In [1]:
import sys
import os
import json

import pandas as pd
import numpy as np

from collections import defaultdict

In [2]:
root_dir = 'C:/Users/delst/OneDrive/Desktop/Code/Workspace/JSON_to_PGSQL'
sys.path.append(root_dir)

from A_Main.Configurations.setup_env import setup_environment
config = setup_environment(root_dir)

In [3]:
raw_data_path = config.raw_data

---

In [4]:
with open(raw_data_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

---

In [5]:
data = raw_data.get('data', {})
posts = data.get('children', [])
post_data = posts[2].get('data', {})
media_data = post_data.get('media', {})
reddit_video_data = media_data.get('reddit_video', {})
reddit_video_data

{'bitrate_kbps': 1200,
 'fallback_url': 'https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?source=fallback',
 'has_audio': True,
 'height': 480,
 'width': 854,
 'scrubber_media_url': 'https://v.redd.it/evmp79fbr56b1/DASH_96.mp4',
 'dash_url': 'https://v.redd.it/evmp79fbr56b1/DASHPlaylist.mpd?a=1689435086%2CZGI5YzZlMTBkMWViZmZjM2E4NjZhODJlMTM3ZjVjMWNhYWI3NjM5YTcyZmE5ZjA2Mjk5OGUyZjQ5OWY3MDdiMQ%3D%3D&amp;v=1&amp;f=sd',
 'duration': 68,
 'hls_url': 'https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3u8?a=1689435086%2CMzA1YjFkMDUwYTIwZDkzZTZjMWVjMGZhYTJmNmI4N2Y0ZWJkYjQ1MjU2YzgwZDJiNmI4MmU0OWRhYTU4MGIyMQ%3D%3D&amp;v=1&amp;f=sd',
 'is_gif': False,
 'transcoding_status': 'completed'}

In [6]:
data = raw_data.get('data', {})
posts = data.get('children', [])

for post in posts:
    post_data = post.get('data', {})
    media_data = post_data.get('media')
    
    if media_data is not None:
        reddit_video_data = media_data.get('reddit_video', {})

reddit_video_data

{'bitrate_kbps': 4800,
 'fallback_url': 'https://v.redd.it/8emliax8966b1/DASH_1080.mp4?source=fallback',
 'has_audio': True,
 'height': 1080,
 'width': 608,
 'scrubber_media_url': 'https://v.redd.it/8emliax8966b1/DASH_96.mp4',
 'dash_url': 'https://v.redd.it/8emliax8966b1/DASHPlaylist.mpd?a=1689435086%2CNGVkYTYxYzk2MzVlODYyZDg0NDIxMzE4MmMzZTlkNjk5MWI0MjdhMmMzZWVkYTA2YzIwZjdkMGM2MjQ3ZTE1NQ%3D%3D&amp;v=1&amp;f=sd',
 'duration': 76,
 'hls_url': 'https://v.redd.it/8emliax8966b1/HLSPlaylist.m3u8?a=1689435086%2CYWJkNzliY2FiODNiNjYwM2YyMDc3YmJiOTU1ZGEyYzA4MDMyYTJmNzE1MTgwNTJhNzdhZWRlNzY3MmMzYzIxZg%3D%3D&amp;v=1&amp;f=sd',
 'is_gif': False,
 'transcoding_status': 'completed'}

---

In [7]:
video_data_store = []
for post in posts:
    post_data = post.get('data', {})
    
    post_id = post_data.get('id', None)
    media_data = post_data.get('media', {})
    
    if media_data and 'reddit_video' in media_data:
        reddit_video = media_data.get('reddit_video', {})
        if isinstance(reddit_video, dict):
            single_data = [
                post_id,
                reddit_video.get('bitrate_kbps'),
                reddit_video.get('fallback_url'),
                reddit_video.get('has_audio'),
                reddit_video.get('height'),
                reddit_video.get('width'),
                reddit_video.get('scrubber_media_url'),
                reddit_video.get('dash_url'),
                reddit_video.get('duration'),
                reddit_video.get('hls_url'),
                reddit_video.get('is_gif'),
                reddit_video.get('transcoding_status'),
            ]
            video_data_store.append(single_data)
video_data_store

[['149yrlt',
  1200,
  'https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?source=fallback',
  True,
  480,
  854,
  'https://v.redd.it/evmp79fbr56b1/DASH_96.mp4',
  'https://v.redd.it/evmp79fbr56b1/DASHPlaylist.mpd?a=1689435086%2CZGI5YzZlMTBkMWViZmZjM2E4NjZhODJlMTM3ZjVjMWNhYWI3NjM5YTcyZmE5ZjA2Mjk5OGUyZjQ5OWY3MDdiMQ%3D%3D&amp;v=1&amp;f=sd',
  68,
  'https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3u8?a=1689435086%2CMzA1YjFkMDUwYTIwZDkzZTZjMWVjMGZhYTJmNmI4N2Y0ZWJkYjQ1MjU2YzgwZDJiNmI4MmU0OWRhYTU4MGIyMQ%3D%3D&amp;v=1&amp;f=sd',
  False,
  'completed'],
 ['149z3aw',
  2400,
  'https://v.redd.it/9az2zw4ou56b1/DASH_720.mp4?source=fallback',
  None,
  720,
  406,
  'https://v.redd.it/9az2zw4ou56b1/DASH_96.mp4',
  'https://v.redd.it/9az2zw4ou56b1/DASHPlaylist.mpd?a=1689435086%2CMjQ5YTAzNzZkMjQxYTNjZjhlNWZjNGFhMjhiMWE2ZDFhZjhlNWEyZjdmNGYwYTFmMTI3N2E2ZDIwODczYzg0Mw%3D%3D&amp;v=1&amp;f=sd',
  58,
  'https://v.redd.it/9az2zw4ou56b1/HLSPlaylist.m3u8?a=1689435086%2CNWQ1OTYyMDFiZWUyNWI1YjFlYTQyZDFmMmNkOTliOTh

In [8]:
for post in posts:
    post_data = post.get('data', {})
    media_data = post_data.get('media', {})
    
    if media_data and 'reddit_video' in media_data:
        reddit_video = media_data.get('reddit_video', {})
        print(reddit_video.keys())

dict_keys(['bitrate_kbps', 'fallback_url', 'has_audio', 'height', 'width', 'scrubber_media_url', 'dash_url', 'duration', 'hls_url', 'is_gif', 'transcoding_status'])
dict_keys(['bitrate_kbps', 'fallback_url', 'height', 'width', 'scrubber_media_url', 'dash_url', 'duration', 'hls_url', 'is_gif', 'transcoding_status'])
dict_keys(['bitrate_kbps', 'fallback_url', 'height', 'width', 'scrubber_media_url', 'dash_url', 'duration', 'hls_url', 'is_gif', 'transcoding_status'])
dict_keys(['bitrate_kbps', 'fallback_url', 'has_audio', 'height', 'width', 'scrubber_media_url', 'dash_url', 'duration', 'hls_url', 'is_gif', 'transcoding_status'])
dict_keys(['bitrate_kbps', 'fallback_url', 'has_audio', 'height', 'width', 'scrubber_media_url', 'dash_url', 'duration', 'hls_url', 'is_gif', 'transcoding_status'])
dict_keys(['bitrate_kbps', 'fallback_url', 'has_audio', 'height', 'width', 'scrubber_media_url', 'dash_url', 'duration', 'hls_url', 'is_gif', 'transcoding_status'])
dict_keys(['bitrate_kbps', 'fallback

In [9]:
columns = ['post_id','bitrate_kbps', 'fallback_url', 'has_audio', 'height', 'width', 'scrubber_media_url', 'dash_url', 'duration', 'hls_url', 'is_gif', 'transcoding_status']
video_data = pd.DataFrame(video_data_store, columns=columns)
video_data

,post_id,bitrate_kbps,fallback_url,has_audio,height,width,scrubber_media_url,dash_url,duration,hls_url,is_gif,transcoding_status
0,149yrlt,1200,https://v.redd.it/evmp79fbr56b1/DASH_480.mp4?s...,True,480,854,https://v.redd.it/evmp79fbr56b1/DASH_96.mp4,https://v.redd.it/evmp79fbr56b1/DASHPlaylist.m...,68,https://v.redd.it/evmp79fbr56b1/HLSPlaylist.m3...,False,completed
1,149z3aw,2400,https://v.redd.it/9az2zw4ou56b1/DASH_720.mp4?s...,None,720,406,https://v.redd.it/9az2zw4ou56b1/DASH_96.mp4,https://v.redd.it/9az2zw4ou56b1/DASHPlaylist.m...,58,https://v.redd.it/9az2zw4ou56b1/HLSPlaylist.m3...,False,completed
2,149wwzb,2400,https://v.redd.it/vstcrxxm756b1/DASH_720.mp4?s...,None,720,406,https://v.redd.it/vstcrxxm756b1/DASH_96.mp4,https://v.redd.it/vstcrxxm756b1/DASHPlaylist.m...,178,https://v.redd.it/vstcrxxm756b1/HLSPlaylist.m3...,False,completed
3,14a36xk,2400,https://v.redd.it/regbdl8au66b1/DASH_720.mp4?s...,False,720,1280,https://v.redd.it/regbdl8au66b1/DASH_96.mp4,https://v.redd.it/regbdl8au66b1/DASHPlaylist.m...,43,https://v.redd.it/regbdl8au66b1/HLSPlaylist.m3...,True,completed
4,149zpnd,1200,https://v.redd.it/0zpvfvyk066b1/DASH_480.mp4?s...,True,394,854,https://v.redd.it/0zpvfvyk066b1/DASH_96.mp4,https://v.redd.it/0zpvfvyk066b1/DASHPlaylist.m...,354,https://v.redd.it/0zpvfvyk066b1/HLSPlaylist.m3...,False,completed
5,149wt3m,4800,https://v.redd.it/3m97pwhg656b1/DASH_1080.mp4?...,True,1080,608,https://v.redd.it/3m97pwhg656b1/DASH_96.mp4,https://v.redd.it/3m97pwhg656b1/DASHPlaylist.m...,10,https://v.redd.it/3m97pwhg656b1/HLSPlaylist.m3...,False,completed
6,149xpc4,2400,https://v.redd.it/fz81vs88g56b1/DASH_720.mp4?s...,None,720,408,https://v.redd.it/fz81vs88g56b1/DASH_96.mp4,https://v.redd.it/fz81vs88g56b1/DASHPlaylist.m...,11,https://v.redd.it/fz81vs88g56b1/HLSPlaylist.m3...,False,completed
7,149vgl6,2400,https://v.redd.it/w1ou6htyr46b1/DASH_720.mp4?s...,True,720,618,https://v.redd.it/w1ou6htyr46b1/DASH_96.mp4,https://v.redd.it/w1ou6htyr46b1/DASHPlaylist.m...,25,https://v.redd.it/w1ou6htyr46b1/HLSPlaylist.m3...,False,completed
8,14a0ozr,4800,https://v.redd.it/8emliax8966b1/DASH_1080.mp4?...,True,1080,608,https://v.redd.it/8emliax8966b1/DASH_96.mp4,https://v.redd.it/8emliax8966b1/DASHPlaylist.m...,76,https://v.redd.it/8emliax8966b1/HLSPlaylist.m3...,False,completed


In [10]:
image_data_store = []

for post in posts:
    preview = post['data'].get('preview', {})
    post_id = post['data'].get('id')

    for image in preview.get('images', []):
        unique_id = image.get('id')

        for item in image.get('resolutions', []):
            single_data = [
                post_id,
                unique_id,
                item.get('url'),
                item.get('width'),
                item.get('height'),
            ]
            image_data_store.append(single_data)

In [11]:
columns = ['post_id', 'image_id', 'url', 'width', 'height']
image_data = pd.DataFrame(image_data_store, columns=columns)
image_data['image_id'] = image_data.image_id.apply(lambda x: x[:-2])
image_data

,post_id,image_id,url,width,height
0,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCa,https://preview.redd.it/urxp0hrc666b1.png?widt...,108,171
1,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCa,https://preview.redd.it/urxp0hrc666b1.png?widt...,216,343
2,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCa,https://preview.redd.it/urxp0hrc666b1.png?widt...,320,508
3,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCa,https://preview.redd.it/urxp0hrc666b1.png?widt...,640,1016
4,14a0ayt,4VWgAl4VJ8GxVz6JYR6o2aXvzunbKRAKU0VefcwCa,https://preview.redd.it/urxp0hrc666b1.png?widt...,960,1524
...,...,...,...,...,...
102,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,216,193
103,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,320,287
104,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,640,574
105,149vuvp,dlXFVWswaCF6GooP3Nh-vUvJ3wacL0XCyygdkQ4l_,https://preview.redd.it/il6cqa38w46b1.jpg?widt...,960,862
